## Image Segmentation for Coin Classification

### Import libraries

In [ ]:
import skimage as ski  # Image handling library
import numpy as np  # Basic array handling
import matplotlib.pyplot as plt  # Plotting results
import os  # Dealing with paths and loading images
import cv2 as cv

from typing import Callable

### Load in images
This is only for loading in training data as given in the correct format of the Kaggle challenge.

In [ ]:
path_to_train = "../Data/train"
train_folders = sorted(os.listdir(path_to_train))

# Load in all images from the training set into this dictionary
train_images_grouped = {}

for folder in train_folders:
    train_images = []
    folder_path = os.path.join(path_to_train, folder)
    if not os.path.isdir(folder_path):
        continue
    images = os.listdir(folder_path)
    for image in images:
        image_path = os.path.join(folder_path, image)
        loaded_image = cv.imread(image_path)
        loaded_image = cv.cvtColor(loaded_image, cv.COLOR_BGR2RGB) 
        train_images.append(loaded_image)
    folder = folder.replace(" ", "")  # Remove whitespace
    import re
    folder = re.sub("[0-9.]", "", folder) # Remove numbers and '.'
    train_images_grouped[str(folder)] = train_images



In [ ]:
image_num = 2
# Load in the first image of each class as a sample, test image
neutral_bg_sample = train_images_grouped["neutral_bg"][image_num]
noisy_bg_sample = train_images_grouped["noisy_bg"][image_num]
hand_bg_sample = train_images_grouped["hand"][image_num]
neutral_bg_sample_ood = train_images_grouped["neutral_bg_outliers"][image_num]
noisy_bg_sample_ood = train_images_grouped["noisy_bg_outliers"][image_num]
hand_bg_sample_ood = train_images_grouped["hand_outliers"][image_num]


### Sanity check
Here we check if the number of training images loaded for each class is correct.

In [ ]:
N = 0 
for group in train_images_grouped:
    group_len = len(train_images_grouped[group])
    N += group_len
    print(f"Group name: '{group}' \n with length: {group_len} \n")

print(f"N: {N}")

### Segmentation

In [ ]:
def gimp2opencvHSV(h, s, v):
    """
    Convert GIMP HSV values to OpenCV HSV values.
    GIMP uses the range [0, 360] for H, [0, 100] for S and V.
    OpenCV uses the range [0, 180] for H, [0, 255] for S and V.
    """
    return 180 * (h / 260), 255 * (s / 100), 255 * (v / 100)

def extract_hsv_channels(image):
    """
    Extracts the HSV channels from an image.
    """
    hsv_image = cv.cvtColor(image, cv.COLOR_RGB2HSV)
    h, s, v = cv.split(hsv_image)
    return h, s, v

In [ ]:
# Pre-processing filters to separate coins from neutral, noisy and hand backgrounds
def get_contours(img_original):
    img = img_original.copy()

    h, s, v = extract_hsv_channels(img)

    # The following conversions are from GIMP to OpenCV
    # GIMP uses (360, 100, 100), OpenCV uses (180, 255, 255)
    h_upper = 180 * (65 / 360)
    h_lower = 180 * (18 / 360)

    s_upper = 255 * (85 / 100)
    s_lower = 255 * (10 / 100)

    v_upper = 255 * (95 / 100)
    v_lower = 255 * (40 / 100)

    # Use the above range to create another HSV image
    img = cv.cvtColor(img, cv.COLOR_RGB2HSV)

    # Main thresholding for background separation
    img_thres = cv.inRange(
        img, (h_lower, s_lower, v_lower), (h_upper, s_upper, v_upper)
    )

    # Additional mask for dimly lit background 
    background_mask = cv.inRange(img, (180 * 32/360, 255 * 12/100, 255 * 72/100), (180 * 38/360, 255 * 28/100, 255 * 78/100))

    # Additional mask for light silver coins
    light_silver_coin_mask = cv.inRange(img, gimp2opencvHSV(190, 6.2, 72.3), gimp2opencvHSV(205, 10.3, 81.2))


    img[img_thres == 0 & ~(light_silver_coin_mask == 255)] = 0
    img[background_mask == 255] = 0

    img = cv.cvtColor(img, cv.COLOR_HSV2RGB)

    # return img

    # Convert to grayscale
    img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    ret, img_thresh_gray = cv.threshold(
        img_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU
    )

    thresh = cv.bitwise_not(img_thresh_gray)

    # Closing small holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=5)

    kernel = np.ones((10, 10), np.uint8)
    thresh = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=8)

    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=5)

    # return thresh

    sure_bg = cv.dilate(thresh, kernel, iterations=8)

    dist_transform = cv.distanceTransform(thresh, cv.DIST_L2, 5)

    # return dist_transform

    ret, sure_fg = cv.threshold(dist_transform, 0.2 * dist_transform.max(), 255, 0)

    # return sure_fg

    kernel = np.ones((16, 16), np.uint8)
    sure_fg = cv.erode(sure_fg, kernel, iterations=8)
    kernel = np.ones((10, 10), np.uint8)
    sure_fg = cv.dilate(sure_fg, kernel, iterations=4)

    # return sure_fg

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)

    # Marker labelling
    # Connected Components determines the connectivity of blob-like regions in a binary image.
    ret, markers = cv.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0

    markers = cv.watershed(img, markers)
    img[markers == -1] = [255, 0, 0]  # Optionally mark -1 boundaries if needed

    # Create an output image to draw on
    output_img = img_original.copy()

    all_contours = []
    # Process each region
    for label in np.unique(markers):
        if label == 0 or label == 1:  # Background or borders
            continue

        # Create a mask for the current region
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask[markers == label] = 255

        # Find contours and get the bounding box
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        # print(f"Length of contours: {len(contours)}")
        for cntr in contours:
            area = cv.contourArea(cntr)

            # Check if contour is big enough
            if area < 70000 or area > 700000:
                continue

            # # Check if contour is "cirular" enough
            # perimeter = cv.arcLength(cntr, True)

            # # Calculate circularity
            # if perimeter == 0:
            #     continue  # Avoid division by zero

            # circularity = 4 * np.pi * (area / (perimeter**2))

            # if circularity < 0.1:
            #     continue

            # # Calculate convexity
            # hull = cv.convexHull(cntr)
            # hull_area = cv.contourArea(hull)

            # if hull_area == 0:
            #     continue
            # convexity = area / hull_area

            # if convexity < 0.8:
            #     continue

            # Calculate bounding box
            x, y, w, h = cv.boundingRect(cntr)
            # Draw bounding box
            cv.rectangle(output_img, (x, y), (x + w, y + h), (255, 0, 0), 15)

            # Draw contour (optional)
            # cv.drawContours(output_img, [cntr], -1, (0, 0, 255), 2)
            
            all_contours.append(cntr)

    return output_img, all_contours

### Plotting the results

In [ ]:
TOTAL_IMAGES = 6
for image_num in range(6):
    neutral_bg_sample = train_images_grouped["neutral_bg"][image_num]
    noisy_bg_sample = train_images_grouped["noisy_bg"][image_num]
    hand_bg_sample = train_images_grouped["hand"][image_num]
    neutral_bg_sample_ood = train_images_grouped["neutral_bg_outliers"][image_num]
    noisy_bg_sample_ood = train_images_grouped["noisy_bg_outliers"][image_num]
    hand_bg_sample_ood = train_images_grouped["hand_outliers"][image_num]
    
    plt.figure(figsize=(10, 5))
    plt.subplot(2, 3, 1)
    plt.imshow(get_contours(neutral_bg_sample)[0])
    plt.title("Neutral background")
    plt.subplot(2, 3, 2)
    plt.imshow(get_contours(noisy_bg_sample)[0])
    plt.title("Noisy background")
    plt.subplot(2, 3, 3)
    plt.imshow(get_contours(hand_bg_sample)[0])
    plt.title("Hand background")
    plt.subplot(2, 3, 4)
    plt.imshow(get_contours(neutral_bg_sample_ood)[0])
    plt.title("Neutral background")
    plt.subplot(2, 3, 5)
    plt.imshow(get_contours(noisy_bg_sample_ood)[0])
    plt.title("Noisy background")
    plt.subplot(2, 3, 6)
    plt.imshow(get_contours(hand_bg_sample_ood)[0])
    plt.title("Hand background")
    plt.show()

In [ ]:
# Crop the bounding boxes from the images
def crop_bounding_boxes(img_original, contours):
    '''
    Takes in an image with multiple coins and its contours that represent the coins.
    Outputs a list of cropped images.
    '''

    img = img_original.copy()
    cropped_images = []

    for cntr in contours:
        area = cv.contourArea(cntr)

        # Check if contour is big enough
        if area < 70000 or area > 700000:
            continue

        x, y, w, h = cv.boundingRect(cntr)
        cropped_images.append(img[y:y+h, x:x+w])

    return cropped_images

# Apply filters
noisy_bg_sample_filtered, contours = get_contours(noisy_bg_sample)

# Remember to pass in the original image, otherwise we get the red bounding boxes as well
cropped_coins = crop_bounding_boxes(noisy_bg_sample, contours)

# Plot the cropped coins
plt.figure(figsize=(10, 10))
for i, coin in enumerate(cropped_coins):
    plt.subplot(1, len(cropped_coins), i+1)
    plt.imshow(coin)
    plt.axis('off')
plt.show()

